# OR case 1

In [25]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np

### 3.

### Constant Data

In [26]:
df = pd.read_csv("data/instance 2.csv")

In [27]:
max_amount = 0
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        if(num > max_amount):
            max_amount = num
            
    temp = row['Stage-2 Machines']
    if(pd.isna(temp) == False):
        x2 = temp.split(',')
        for element in x2:
            num = int(element)
            if(num > max_amount):
                max_amount = num


In [28]:
# max_amount
M = max_amount

In [29]:
# instance i 中 job 的總數量
I = len(df) # order num
S = 2 # stage num


orders = range(0, I)
machines = range(0, M)
stages = range(0, S)


In [30]:
P = []
for index, row in df.iterrows():
    P.append([])
    P[index].append(row['Stage-1 Processing Time'])
    P[index].append(row['Stage-2 Processing Time'])

In [31]:
P

[[2.7, 1.5],
 [1.6, 2.3],
 [1.0, 2.7],
 [2.8, 0.8],
 [0.8, 1.9],
 [2.7, 0.0],
 [1.4, 1.5],
 [2.2, 0.0],
 [0.8, 1.8],
 [2.2, 2.2],
 [2.5, 1.5]]

In [32]:
# C_ijm -> init 3 dim 都是 false 的東西 I * 2 * M
C =  []
for i in range(I):
    C.append([])
    C[i].append([0 for i in range(M)])
    C[i].append([0 for i in range(M)])

In [33]:
for index, row in df.iterrows():
    x1 = row['Stage-1 Machines'].split(',')
    for element in x1:
        num = int(element)
        C[index][0][num-1] = 1

        temp = row['Stage-2 Machines']
        if(pd.isna(temp) == False):
            x2 = temp.split(',')
            for element in x2:
                num = int(element)
                C[index][1][num-1] = 1


In [34]:
C[0][0]

[1, 1, 1, 1, 1]

In [35]:
D = []
for index, row in df.iterrows():
    D.append(row['Due Time'])
D

[10, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10]

### Decision variable – Delay

In [36]:
Q3_delay = Model('Q3-delay')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q3_delay.update()

In [37]:
l

{0: <gurobi.Var l_0>,
 1: <gurobi.Var l_1>,
 2: <gurobi.Var l_2>,
 3: <gurobi.Var l_3>,
 4: <gurobi.Var l_4>,
 5: <gurobi.Var l_5>,
 6: <gurobi.Var l_6>,
 7: <gurobi.Var l_7>,
 8: <gurobi.Var l_8>,
 9: <gurobi.Var l_9>,
 10: <gurobi.Var l_10>}

### Setting the objective function

In [38]:
Q3_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [39]:
Q3_delay.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_delay.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_delay.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_delay.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')


Q3_delay.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints machines')
Q3_delay.update()

In [40]:
Q3_delay.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2376 rows, 627 columns and 11286 nonzeros
Model fingerprint: 0xa01381f0
Variable types: 22 continuous, 605 integer (605 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+02]
Found heuristic solution: objective 4.0000000
Presolve removed 132 rows and 264 columns
Presolve time: 0.05s
Presolved: 2244 rows, 363 columns, 11179 nonzeros
Variable types: 22 continuous, 341 integer (341 binary)

Root relaxation: objective 0.000000e+00, 192 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0    0.00000    0

In [41]:
a = Q3_delay.objVal
print("objective value =", a)

objective value = 0.0


### Decision variable – makespan

In [42]:
Q3_makespan = Model('Q3-makespan')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q3_makespan.update()

### Setting the objective function

In [43]:
Q3_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [44]:
Q3_makespan.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_makespan.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_makespan.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_makespan.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
# C_ism
Q3_makespan.addConstrs((y[i][s][m] <= (1 if C[i][s][m] == 0 else 1) for i in orders for s in stages for m in machines), 'constraints for cook machines')
Q3_makespan.addConstrs((w >= x[i][1] for i in orders), 'makespan')
Q3_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')
# Q3_makespan.addConstr((w == 6.1), 'delay num')

Q3_makespan.update()

In [45]:
Q3_makespan.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2388 rows, 628 columns and 11319 nonzeros
Model fingerprint: 0x6e0902c4
Variable types: 23 continuous, 605 integer (605 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+02]
Presolve removed 214 rows and 282 columns
Presolve time: 0.01s
Presolved: 2174 rows, 346 columns, 10804 nonzeros
Variable types: 23 continuous, 323 integer (323 binary)

Root relaxation: objective 4.400000e+00, 148 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.40000    0   14          -    4.40000      -     -    0s
     0     0    4.40000    0   28          -    4.40000      -     -    0s

In [46]:
print("objective value =", Q3_makespan.objVal)

objective value = 7.500000000000009


## Results

In [47]:
x_opt = {}
for i in orders:
    x_opt[i] = {}
    for s in stages:
        x_opt[i][s] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for s in stages:
        y_opt[i][s] = {}
        for m in machines:
            y_opt[i][s][m] = {}

for var in Q3_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])][int(index[2])] = var.x

## Plot Gantt Charts

In [48]:
gantt_plot_2_3(x_opt, P, y_opt, instance_no, 3)

NameError: name 'gantt_plot_2_3' is not defined